# Python y empresa: Introducción a los problemas básicos de optimización 

** Daniel Domene-López ** y ** Carlos Planelles-Alemany ** para [PyConES16](http://2016.es.pycon.org/) - Almería 

## 0. Indice 
> 1. Introducción.
> 2. Instalación Pyomo.
> 3. El problema de la mochila (Knapsack Problem). 
> 4. El problema del viajante de comercio. 

## 1. Introducción.

La optimización de procesos industriales ha experimentado un crecimiento considerable durante los últimos años. las industrias modernas apuestan por ella para mejorar la producción, reducir los costes, disminuir el impacto ambiental e incrementar la seguridad de los procesos. en este sentido, es sensata la introducción y formación de los nuevos ingenieros en esta rama de la ciencia.

Resulta curioso pensar que en ámbito de las ciencias de la computación, la optimización suela hacer referencia a la reducción en el tiempo de ejecución del programa. Pero la optimización matemática (también conocida como investigación de operaciones o programación matemática) no tiene nada que ver con este problema y, en muchos casos, la formación se recibe en las universidades no pasa de enunciar y explicar el algoritmo SIMPLEX para la resolución de problemas lineales con variables continuas. 

A grandes rasgos, la optimización consite en maximizr o minimiza una función real, eligiendo sistemáticamente valores de entrada, siempre tomados de un conjunto permitido, y calculando el valor de la función. Este problema se torna interesante y complejo cuando aparecen restricciones, que pueden ser lineales o no lineales, además de variables enteras, empleadas para modelar la toma de decisiones discretas. 

Multitud de problemas de optimización en las empresas pueden llegar a expresarse a partir de un pequeño conjunto de problemas de optimización sencillos, como lo es el ** problema de la mochila (Knapsack Problem)** o el ** problema del viajante **.
Estos problemas aparecen, en muchas ocasiones de manera indirecta, en problemas relacionados con la organización de la logística de la empresa o en la toma de decisiones sobre mercaderías e incluso en la elección de proyectos de I+D+i. 
El estudio y buen concoimiento de los problemas básicos de optimización es básico para comprender y abordar problemas de una magnitud superior, además de ser un magnífico punto de partida para introducirse en la sintaxis del modelado algebraico. 

Existen varios lenguajes de modelado algebráico comerciales. Dos de los más utilizados para resolver este tipo de problemas son GAMS y AMPL. En Python existen varias bibliotecas que permiten resolver el mismo tipo de probleams (CVXOpt, CVXPy, PulP, OpenOpt o Pyomo). 

De todos ellos, Pyomo resulta interesante porque:
* Es **Open Source**, desarrollado por Sania Lab. 
* Integración de **Solvers**.
* Es un **lenguaje de programación** en si mismo, lo que lo hace un lenguaje robusto, con una extensa documentación...además de soportar características de programación moderna, como las clases y las funciones. 

En los siguientes puntos mostramos como realizar la instalación de Pyomo y cómo resolver el ** Problema de la Mochila ** y el ** Problema del viajante de comercio **. 

## 2. Instalación Pyomo.

Para la instalación de Pyomo recomendamos seguir las guías que aparecen en el Notebook ** [Guía de instalación de Pyomo](https://github.com/CAChemE/pyomo-instalacion) ** en el GitHub de ** [CAChemE](http://cacheme.org/)**. En este Notebook se encuentra toda la información necesaria para instalar Pyomo y los solvers necesarios para resolver los problemas de una manera muy sencilla gracias al trabajo de 
**[ Juan Luis Cano](https://www.linkedin.com/in/juanluiscanor) **. 

Aquí un pequeño resumen. En Python 3.5 teclear en la terminal:

*conda install pyomo -c conda-forge *

Para instalar el solver glpk:

*conda install glpk -c conda-forge*

## 3. El problema de la Mochila (Knapsack Problem). 

El problema de la mochila es un problema de optimización combinatoria, busca la mejor solución entre un conjunto finito de posibles soluciones a un problema. 
Cuando se proporciona un volumen fijo, el cual debe ser llenado con objetos valiosos de volumen específico se trata de encontrar un programa que permita la selección de los objetos que hagan máximo el valor de la mochila.  

$$
\begin{array}{lll}
\max & f(x) = \sum_{i=1}^F c_i x_i & \\
& \\\
s.t. & \sum_{i=1}^F  V_{ij} x_i \leq V_{max} & \forall i = 1 \ldots F\ & \\\
& \\\
     & \sum_{i=1}^F  M_{ij} x_i \leq M_{max} & \forall i = 1 \ldots F\ & \\\
     & \\\
     & x_i \geq 0 & \forall i = 1 \ldots F
\end{array}
$$

*** Ejemplo *** : De entre los siguients objetos, escoge los que puedes introducir en la tu mochila, teniendo en cuenta que han de cumplirse las restricciones de capacidad y peso máximos, amén de que la suma del valor de los objetos introducidos en lam ochila ha de ser máxima. 

*** Datos ***: A continuación el archivo con los datos del problema. 

In [1]:
! cat Knapsack.dat 

*** Resolución ***: En un archivo que llamaremos Knapsack.py introduciremos el siguiente código para resolver el problema. A continuación se muestra el contenido del archivo. 

In [2]:
from pyomo.environ import *

# 1. Generamos el modelo Abstracto, pues le introduciremos los datos de manera externa con un archivo de datos.
model = AbstractModel() 

# 2. Generamos el Set de los objetos del problema. 
model.ITEMS = Set() 

# 3. Asignamos los valores del archivo de datos a los coeficientes del modelo. 
# VOLUMENES
model.v = Param(model.ITEMS, within = PositiveReals)
# MASAS
model.w = Param(model.ITEMS, within = PositiveReals)
# VALOR DE LOS OBJETOS
model.c = Param(model.ITEMS, within = PositiveReals)
# UNIDADES DE CADA OBJETO DISPONIBLES
model.u = Param(model.ITEMS, within = PositiveReals) 
# LIMITE DE VOLUMEN
model.limitV = Param(within = PositiveReals)
# LIMITE DE MASA
model.limitW = Param(within = PositiveReals) 

# 4. Generamos la variable del problema x (unidades de objetos que introducimos a la mocila)
model.x = Var(model.ITEMS, within = NonNegativeIntegers) 

Luego de introducir los valores de los coeficientes y declarar las variables ha de establecerse las ecuaciones del modelo. 

** Función Objetivo ** 

$$
\begin{array}{111}
\max & f(x) = \sum_{i=1}^F c_i x_i & \\
 & \
     & \
     & 
\end{array}
$$

In [4]:
def value_rule(model):
    
    return sum(model.c[i] * model.x[i]  for i in model.ITEMS)

model.value = Objective(sense = maximize, rule=value_rule)

** Restricción de Volumen ** 

$$
\begin{array}{lll}
\\\
s.t. & \sum_{i=1}^F  V_{ij} x_i \leq V_{max} & \forall i = 1 \ldots F\ & \\\
     & \\
     & 
\end{array}
$$

In [5]:
def volum_rule(model):
    
    return sum(model.v[i] * model.x[i] for i in model.ITEMS) <= model.limitV

model.volum = Constraint( rule = volum_rule )

** Restricción de Masa **

$$
\begin{array}{lll}
\
s.t. & 
     & \sum_{i=1}^F  M_{ij} x_i \leq M_{max} & \forall i = 1 \ldots F\ & \\\
     & 
\end{array}
$$

In [6]:
def weight_rule(model): 
    
    return sum(model.w[i] * model.x[i] for i in model.ITEMS) <= model.limitW 

model.weight = Constraint( rule = weight_rule )

En el enunciado del modelo no se ha hecho referencia a la cantidad de objetos que se pueden escoger. Es decir, se puede escoger más de un anillo, por lo que el valor de su variable podrá ser 0, 1, 2... hasta 10. Esto también es una restricción, por lo que ha de introducirse explícitamente. 

** Restricción de cantidad de objeto ** 

$$
\begin{array}{lll}
\
s.t. & 
     & \ x_i \leq U_{i} & \forall i = 1 \ldots F\ & \\\
     & 
\end{array}
$$

Como puedes observa, la siguiente condición implica un número de ecuaciones igual al número de objetos entre los que se selecciona. Para una pequeña cantidad de los mismos podría ser una opción introducirlas de una en una, pero existe la posibilidad de escribirlas todas a la vez, aprovechando el set que hemos generado.

In [7]:
def amount_rule(model, i):
    
    return model.x[i] <= model.u[i]

model.amount = Constraint( model.ITEMS, rule = amount_rule)

El archivo Knapsack.py quedaría por tanto:

In [8]:
! cat Knapsack.py 

Para resolver basta con teclear el siguiente comando, en el que indicamos a pyomo que ha de resolver el modelo en el arvhivo .py con los datos del archivo .dat y utilizando el solver glpk. 

In [10]:
!pyomo solve --solver=glpk Knapsack.py Knapsack.dat 

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.11] Applying solver
[    0.15] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 232.0
    Solver results file: results.yml
[    0.16] Applying Pyomo postprocessing actions
[    0.16] Pyomo Finished


Se genera entonces un archivo .json con los resultados de la optimización. 

In [11]:
! cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 232.0
  Upper bound: 232.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 11
  Number of nonzeros: 31
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.033164024353
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------

De esta manera, en la mochila se deberan introducir:
* 7 Brazaletes.
* 1 Diamante.
* 10 Anillos.
* 1 Azafrán. 

para obtener un valor máximo de 232 Dinares. 

## 4. El Problema del Viajante de Comercio. 

El problema del viajante de comercio tiene como objetivo unir un conjunto de puntos recorriendo para ello el menor espacio posible. Es un problema típico en organización comercial, pues se utilizada para optimizar rutas de viaje. El modelo matemático del problema se expone a continuación:

$$
\begin{array}{lll}
\min & f(x) = \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} y_{ij} \:\:\:\: i,j = 1,\cdots, n \ i=j & \\\
& \\\
s.t. & \sum_{i=1, i \neq j}^{n} y_ {ij} = 1 \:\:\:\: i,j = 1,\cdots, n \ i=j & \\\
     & \\\
     & \sum_{j=0, i \neq j}^{n} y_ {ij} = 1 \:\:\:\: i,j = 1,\cdots, n \ i=j && \\\
     & \\\
     & u_i-u_j+n y_{ij} \leq n-1 \:\:\:\: 1 \leq i \neq j \leq n & \\\
     & \\\
     & y_{ij} = [0 , 1]
\end{array}
$$

**Ejemplo**: Una empresa tiene clientes repartidos por toda España y quiere que uno de sus comerciantes visite a cada uno de ellos. Por supuesto la empresa quiere que el gasto de combustible sea el menor posible, por lo que considera la determinación de una ruta óptimza que asegure que el comerciante realizará el número mínimo de kilómetros. Determinar la ruta. 

In [13]:
! cat tpsData.dat



set I := Madrid Barcelona Almeria Valencia Vigo Murcia Salamanca Zaragoza Toledo Sebastian Santander Valladolid Granada Sevilla Badajoz Castellon Cuenca Tarragona Lerida Huelva Soria;

param n := 21;

param A :
             Madrid Barcelona Almeria Valencia Vigo Murcia Salamanca Zaragoza Toledo Sebastian Santander Valladolid Granada Sevilla Badajoz Castellon Cuenca Tarragona Lerida Huelva Soria   :=
  Madrid     0      621       535     355      602  397    220       314      72     453       425       196        420     534     404     420       166    545       461    616    228
  Barcelona  621    0         790     351      1153 594    845       313      692    571       708       727        888     996     1022    280       541    100       163    1090   467
  Almeria    535    790       0       424      1140 204    758       752      493    991       978       733        162     407     616     507       483    691       745    503    763
  Valencia   355    351       424     0 

<img src="viajante_planteamiento.jpg" alt="ejemplo" style="width: 750px;"/>

**Resolución:** en un archivo tps.py introduciremos el siguiente código.

In [14]:
from pyomo.environ import *

# 1. Generamos el modelo Abstracto, pues le introduciremos los datos demanera externa con un archivo de datos. 
model = AbstractModel()

# 2. Generamos el Set que contiene las ciudades que visitará el comerciante
model.I = Set(ordered=True) #ciudades

# 3. Asignamos los valores del archivo de datos a los coeficientes del modelo. 
# Distancia entre las ciudades
model.A = Param(model.I, model.I)
# Número de ciudades
model.n = Param()

# 4. Generamos las variables del problema y ( si y = 1 -> existe la ruta entre las ciudades)
model.y = Var(model.I, model.I, within=Binary)
# Variable auxiliar para romper ciclos. 
model.u = Var(model.I) 

** Función Objetivo **

$$ min \sum_{i=0}^{n} \sum_{j=0}^{n} c_{ij} y_{ij} \:\:\:\: i,j = 0,\cdots, n \ i=j $$

In [15]:
def _objfunc(model):
    
    return summation(model.A, model.y)

model.OBJ = Objective(rule=_objfunc, sense=minimize)

$$\sum_{i=0, i \neq j}^{n} y_ {ij} = 1$$

In [16]:
def const1(model, i):
    
    return sum(model.y[i,j] for j in model.I if (i != j)) == 1

model.CONST1 = Constraint(model.I, rule=const1)

$$ \sum_{j=0, i \neq j}^{n} y_ {ij} = 1 $$

In [17]:
def const2(model, j):
    
    return sum(model.y[i,j] for i in model.I if (i != j)) == 1

model.CONST2 = Constraint(model.I, rule=const2)

$$ u_i-u_j+n y_{ij} \leq n-1 \:\:\:\: 1 \leq i \neq j \leq n $$

In [18]:
def const3(model, i, j):
    
    if (i == model.I[1] or j == model.I[1] or j == i):
        
        return Constraint.Skip
    
    else:
        
        return model.u[i] - model.u[j] + model.n*model.y[i,j] <= model.n-1
    
model.CONST3 = Constraint(model.I, model.I, rule=const3)

**Resolver**

In [19]:
! pyomo solve tps.py tpsData.dat --solver=glpk

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
A
[    0.00] Creating model
[    0.10] Applying solver
[    0.63] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 4190.0
    Solver results file: results.yml
[    0.65] Applying Pyomo postprocessing actions
[    0.65] Pyomo Finished


**Resultados**

In [20]:
! cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4190.0
  Upper bound: 4190.0
  Number of objectives: 1
  Number of constraints: 423
  Number of variables: 441
  Number of nonzeros: 1981
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 411
      Number of created subproblems: 411
  Error rc: 0
  Time: 0.454398155212
# ----------------------------------------------------------
#   Solution Information
# -----------------------------

<img src="viajante_solucion.jpg" alt="ejemplo" style="width: 750px;"/>

## Bonus Track  (DAE Toolbox)!! : 

DAE Toolbox permite a los usuarios trabajar con ecuaciones diferenciales en pyomo, tanto ecucaiones diferenciales ordinarias (EDO), como parciales (EDP). Veamos un ejemplo a continuación.

### Simulación de el llenado de un tanque de agua


$$ A \frac{dh}{dt} = K_{1}a_{1} - K_2 a_2 \sqrt{2gh} $$

$$ h(0) = 0.5 m $$

$$ 0 \leq t \leq 200 s$$

<img src="deposito.jpg" alt="ejemplo" style="width: 750px;"/>

In [23]:
! cat tankSimulation.py

# Required imports
from pyomo.environ import *
from pyomo.dae import *
from pyomo.dae.plugins.finitedifference import Finite_Difference_Transformation

m = ConcreteModel()

m.K1 = Param(initialize=0.05)  #m3/s
m.K2 = Param(initialize=0.015) #m3/s
m.a1 = Param(initialize=0.6)
m.a2 = Param(initialize=0.5)
m.A = Param(initialize=0.5)    #m2
m.g = Param(initialize=10.0)    #m2/s

T = 200.0

#variables del problema
m.t = ContinuousSet(bounds=(0, T))
m.h = Var(m.t, within = NonNegativeReals, initialize = 0.4)

#variables derivadas respecto a las variables independientes x t
m.dhdt = DerivativeVar(m.h, wrt=m.t)

#restricciones que se deben cumplir
def _bdm(m, j): #balance de materia
    if j == 0 :
        return Constraint.Skip
    return m.A*m.dhdt[j] == m.K1*m.a1-m.K2*m.a2*sqrt(2*m.g*m.h[j])
m.bdm = Constraint(m.t, rule=_bdm)

def _initcon(m, j):
    if j != 0:
        return Constraint.Skip
    return m.h[j] == 0.5
m.initcon = Constraint(m.t,  rule=_initcon)

#funcion objetivo "dummy"
m.o

In [24]:
!python tankSimulation.py



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      101
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:       41
                     variables with only lower bounds:       21
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ